# Group by attack

In [1]:
# load result file
import pandas as pd

result_file = '/nvme1/hungdx/Lightning-hydra/logs/results/TTA_benchmark_test_det_show_lts/ToP_LA19/ADV_2025_cnsl_xlsr_vib_paper_ToP_LA19.txt'
df = pd.read_csv(result_file, sep=' ', header=None)
df.columns = ['utt', 'spoof', 'score']


# meta datafile
metadata_file = '/nvme1/hungdx/Lightning-hydra/data/TTA_benchmark_2025/ADV_2025/ADV/protocol.txt'
metadata = pd.read_csv(metadata_file, sep=' ', header=None)
metadata.columns = ['utt', 'label', 'attack_model', 'attack_type']


# merge df and metadata
df = pd.merge(df, metadata, on='utt', how='left')

# print the first 5 rows
print(df.head())
# print the last 5 rows

                                      utt     spoof     score     label  \
0  ASVspoof2021_DF_eval/DF_E_2814598.flac -2.331280  2.107358  bonafide   
1  ASVspoof2021_DF_eval/DF_E_4998650.flac -2.310411  2.088900  bonafide   
2  ASVspoof2021_DF_eval/DF_E_4192130.flac -2.393897  2.160999  bonafide   
3  ASVspoof2021_DF_eval/DF_E_2592924.flac -2.411914  2.170061  bonafide   
4  ASVspoof2021_DF_eval/DF_E_3631097.flac -2.352637  2.124310  bonafide   

  attack_model attack_type  
0         none        none  
1         none        none  
2         none        none  
3         none        none  
4         none        none  


# Performance group by attack_type & attack_model

In [4]:
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score
import eval_metrics_DF as em

def calculate_group_eer(df, group_column='attack_type'):
    """Calculate EER for each group in the dataset."""
    results = {}
    
    # Overall EER calculation
    bona_scores = df[df['label'] == 'bonafide']['score'].values
    spoof_scores = df[df['label'] == 'spoof']['score'].values
    
    if len(bona_scores) > 0 and len(spoof_scores) > 0:
        overall_eer, overall_threshold = em.compute_eer(bona_scores, spoof_scores)
        results['overall'] = {
            'eer': overall_eer * 100,
            'threshold': overall_threshold,
            'samples': len(df)
        }
    
    # Group-specific EER calculation
    group_results = {}
    all_bona_scores = df[df['label'] == 'bonafide']['score'].values
    
    for group_name, group_df in df.groupby(group_column):
        if group_name in ['none', '-']:  # Skip bonafide group
            continue
            
        # Get spoof samples for this attack
        group_spoof_scores = group_df[group_df['label'] == 'spoof']['score'].values
        
        if len(group_spoof_scores) > 0 and len(all_bona_scores) > 0:
            # Calculate EER for this specific attack vs all bonafide
            group_eer, group_threshold = em.compute_eer(all_bona_scores, group_spoof_scores)
            
            group_results[group_name] = {
                'eer': group_eer * 100,
                'threshold': group_threshold,
                'samples': len(group_df),
                'spoof_samples': len(group_spoof_scores)
            }
    
    results['groups'] = group_results
    return results

# Use with your existing df
results_by_type = calculate_group_eer(df, 'attack_type')
results_by_model = calculate_group_eer(df, 'attack_model')

# Print results
print("EER by Attack Type:")
for group, metrics in results_by_type['groups'].items():
    print(f"{group}: {metrics['eer']:.2f}%")

print("\nEER by Attack Model:")
for group, metrics in results_by_model['groups'].items():
    print(f"{group}: {metrics['eer']:.2f}%")

EER by Attack Type:
BIM: 2.43%
FGSM: 14.85%
PGD: 14.65%

EER by Attack Model:
aasistssl: 13.65%
conformerssl: 10.06%
